In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd
from scipy.stats import ttest_ind
from numba import njit, jit
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [2]:
meta = pd.read_csv('../data/META_SL.csv')

In [3]:
meta['plat'].unique()

array(['Agilent', 'Illumina_NGS', 'Affymetrix_U133', 'Illumina_Chip',
       'Affymetrix_HUG', 'Affymetrix_PRV'], dtype=object)

In [4]:
meta.head()

,CN,type,plat,TP
0,C_1,Solid tumor,Agilent,Solid tumor_Agilent
1,C_2,Solid tumor,Agilent,Solid tumor_Agilent
2,C_3,Solid tumor,Agilent,Solid tumor_Agilent
3,C_4,Solid tumor,Agilent,Solid tumor_Agilent
4,C_5,Solid tumor,Agilent,Solid tumor_Agilent


#### Данные, на которых нужно проверить гипотезу

In [5]:
data = pd.read_csv('../data/50.csv')

In [6]:
data.head()

,S9bcbf828_bd0c_4a71_8021_7a262de8978c,S23ea1a02_938c_47cc_8a7d_abd33db1f305,Sddfe7bdf_0d22_4f98_b712_09df67794990,Sb80aec3c_742c_4e9d_b45b_53809b6b80f1,S988b3794_2640_4ff6_a739_d473adbca3e8,S5d095a36_74db_44ec_bca2_35af7a47f1c5,S97e6b861_062c_4235_8f12_db621f22e353,S9120d8d8_8605_493d_a963_7642852778c4,S10aa11ee_6df1_42f6_bc32_6ffed2e7d25c,S3e1fc047_a69e_469b_a005_33b208fdbfd3,...,AFX_GPL16043_B_3,AFX_GPL16043_B_4,AFX_GPL16043_C_1,AFX_GPL16043_C_2,AFX_GPL16043_C_3,AFX_GPL16043_C_4,AFX_GPL16043_D_1,AFX_GPL16043_D_2,AFX_GPL16043_D_3,AFX_GPL16043_D_4
A2M,7780.567009,5649.049966,5521.783630,11224.365025,22027.253894,10980.433967,6970.974377,7455.672252,7438.377210,3852.802446,...,21523.164457,21298.285650,13244.530619,12198.683792,13250.227027,13833.029054,10411.925636,9940.468020,10515.748423,11124.930431
AACS,2077.267884,1813.223010,1942.758075,2063.804726,1804.612723,1590.917565,1642.354474,1823.025000,2104.443884,2105.910908,...,464.983896,456.512954,439.422416,312.676776,395.721569,443.746235,489.037088,382.572034,475.812045,406.091008
AAK1,219.198828,223.767089,126.758979,121.974333,162.827447,116.404527,109.746882,139.339871,117.041911,266.320006,...,7601.859351,7108.477677,4274.603365,4041.261171,3463.077996,3804.967294,8835.360555,10240.859421,9910.532272,11504.603396
AAMP,3714.612989,3209.826230,3324.890473,3746.213653,4941.364558,3170.299372,3276.375094,3154.220565,4166.652447,5900.787951,...,3817.166833,3917.644445,2719.680369,2651.442744,2622.390482,2835.039290,2692.080831,2796.198834,2812.978561,2848.497722
AASDHPPT,1687.042609,2207.237340,1970.491296,1451.807008,1360.936758,2267.317241,1997.760243,2358.597424,1658.658083,1070.783111,...,9360.850354,8955.214628,4790.496472,4964.339277,4678.400590,4302.749726,7160.727549,7081.723208,7089.783991,7176.928726


In [7]:
data.columns = meta['CN']
data = data.astype(float)

In [8]:
data.head()

CN,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_7158,C_7159,C_7160,C_7161,C_7162,C_7163,C_7164,C_7165,C_7166,C_7167
A2M,7780.567009,5649.049966,5521.783630,11224.365025,22027.253894,10980.433967,6970.974377,7455.672252,7438.377210,3852.802446,...,21523.164457,21298.285650,13244.530619,12198.683792,13250.227027,13833.029054,10411.925636,9940.468020,10515.748423,11124.930431
AACS,2077.267884,1813.223010,1942.758075,2063.804726,1804.612723,1590.917565,1642.354474,1823.025000,2104.443884,2105.910908,...,464.983896,456.512954,439.422416,312.676776,395.721569,443.746235,489.037088,382.572034,475.812045,406.091008
AAK1,219.198828,223.767089,126.758979,121.974333,162.827447,116.404527,109.746882,139.339871,117.041911,266.320006,...,7601.859351,7108.477677,4274.603365,4041.261171,3463.077996,3804.967294,8835.360555,10240.859421,9910.532272,11504.603396
AAMP,3714.612989,3209.826230,3324.890473,3746.213653,4941.364558,3170.299372,3276.375094,3154.220565,4166.652447,5900.787951,...,3817.166833,3917.644445,2719.680369,2651.442744,2622.390482,2835.039290,2692.080831,2796.198834,2812.978561,2848.497722
AASDHPPT,1687.042609,2207.237340,1970.491296,1451.807008,1360.936758,2267.317241,1997.760243,2358.597424,1658.658083,1070.783111,...,9360.850354,8955.214628,4790.496472,4964.339277,4678.400590,4302.749726,7160.727549,7081.723208,7089.783991,7176.928726


In [9]:
## Названия колонок, которые относятся к конкретному типу платформы

plat_1 = meta[meta['plat'] == 'Agilent']['CN']
plat_2 = meta[meta['plat'] == 'Affymetrix_U133']['CN']

In [10]:
print("Всего в файле должна получиться " + str(plat_1.shape[0] +plat_2.shape[0])  + " колонка")

Всего в файле должна получиться 3091 колонка


In [11]:
## Делаем два набора данных, чтобы к каждому добавить свою метку ("T" и "N")

Agilent = data[plat_1]
Affymetrix_U133 = data[plat_2]

In [12]:
## Функция создания ряда с меткой s и размера size

def create_mark_row(s, size):
    row = []
    
    for i in range(size):
        row.append(s)
        
    return row

### Agilent

In [13]:
Agilent.head()

CN,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_4548,C_4549,C_4550,C_4551,C_4552,C_4553,C_4554,C_4555,C_4556,C_4557
A2M,7780.567009,5649.049966,5521.783630,11224.365025,22027.253894,10980.433967,6970.974377,7455.672252,7438.377210,3852.802446,...,11447.562056,15218.680180,11742.456640,11494.912173,12364.914160,16573.636815,12966.771709,14347.485737,16754.687301,15082.744534
AACS,2077.267884,1813.223010,1942.758075,2063.804726,1804.612723,1590.917565,1642.354474,1823.025000,2104.443884,2105.910908,...,1253.437597,1576.743505,1265.634620,1313.410066,1574.618301,1458.181913,1615.729658,1614.399387,1575.464694,1700.073344
AAK1,219.198828,223.767089,126.758979,121.974333,162.827447,116.404527,109.746882,139.339871,117.041911,266.320006,...,143.342364,101.692552,138.145724,136.642919,78.123826,100.597968,82.829049,92.385713,139.674752,97.794977
AAMP,3714.612989,3209.826230,3324.890473,3746.213653,4941.364558,3170.299372,3276.375094,3154.220565,4166.652447,5900.787951,...,2737.106909,3277.135389,2733.147280,2768.925412,3499.023808,3092.273376,3509.176563,3028.535442,3780.043258,3400.193029
AASDHPPT,1687.042609,2207.237340,1970.491296,1451.807008,1360.936758,2267.317241,1997.760243,2358.597424,1658.658083,1070.783111,...,2467.661628,2758.033035,2297.264480,2249.109982,2006.334164,2860.130130,1973.569087,2459.377978,2418.045870,2516.618253


In [14]:
new_pd = pd.DataFrame(columns=Agilent.columns)
new_pd.loc["SYMBOL"] = create_mark_row("N", 1658)

In [15]:
new_pd.head()

CN,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_4548,C_4549,C_4550,C_4551,C_4552,C_4553,C_4554,C_4555,C_4556,C_4557
SYMBOL,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N


In [16]:
Agilent = new_pd.append(Agilent)

In [17]:
Agilent.head()

CN,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_4548,C_4549,C_4550,C_4551,C_4552,C_4553,C_4554,C_4555,C_4556,C_4557
SYMBOL,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
A2M,7780.57,5649.05,5521.78,11224.4,22027.3,10980.4,6970.97,7455.67,7438.38,3852.8,...,11447.6,15218.7,11742.5,11494.9,12364.9,16573.6,12966.8,14347.5,16754.7,15082.7
AACS,2077.27,1813.22,1942.76,2063.8,1804.61,1590.92,1642.35,1823.03,2104.44,2105.91,...,1253.44,1576.74,1265.63,1313.41,1574.62,1458.18,1615.73,1614.4,1575.46,1700.07
AAK1,219.199,223.767,126.759,121.974,162.827,116.405,109.747,139.34,117.042,266.32,...,143.342,101.693,138.146,136.643,78.1238,100.598,82.829,92.3857,139.675,97.795
AAMP,3714.61,3209.83,3324.89,3746.21,4941.36,3170.3,3276.38,3154.22,4166.65,5900.79,...,2737.11,3277.14,2733.15,2768.93,3499.02,3092.27,3509.18,3028.54,3780.04,3400.19


### Affymetrix_U133

In [18]:
Affymetrix_U133.head()

CN,C_780,C_781,C_782,C_783,C_784,C_785,C_786,C_787,C_788,C_789,...,C_5605,C_5606,C_5607,C_5608,C_5609,C_5610,C_5611,C_5612,C_5613,C_5614
A2M,15244.037597,21656.502375,21376.407090,27586.697175,11280.775571,15715.328460,6170.718325,28239.297270,9387.326207,21515.417155,...,170.319532,488.004578,281.182602,406.116201,89.440637,268.346496,517.440817,508.630392,80.286921,388.482511
AACS,3312.266709,1020.928005,687.675494,858.281322,1736.041395,1318.669812,1355.925338,638.935306,2172.844845,1553.793602,...,1042.849687,672.899861,923.000893,924.501364,1135.674020,880.556050,758.422583,788.322474,792.258138,600.134995
AAK1,666.384152,261.522094,429.121472,261.704281,187.638173,294.833175,429.513820,341.132207,930.045341,528.938901,...,748.078145,276.207751,299.855225,276.199409,620.600524,562.018822,377.929045,490.827983,2076.457680,355.812548
AAMP,2134.545567,2351.486706,1849.311966,2603.454053,2685.111724,1907.242382,3650.811882,2530.545838,1799.862865,2050.178933,...,1738.711552,2212.359858,1760.626010,1712.421240,2376.794202,2120.278158,1668.201413,1141.020630,3546.347129,2025.645351
AASDHPPT,3567.487401,2641.235612,1969.401886,2610.263919,3801.580391,3217.819767,1975.322974,2885.938171,3805.491005,4356.250949,...,1906.136645,1460.234527,1861.546813,1009.483910,1302.165530,1611.778250,1333.114198,1161.699415,2169.200352,2703.364512


In [19]:
new_pd = pd.DataFrame(columns=Affymetrix_U133.columns)
new_pd.loc["SYMBOL"] = create_mark_row("T", 1433)

In [20]:
new_pd.head()

CN,C_780,C_781,C_782,C_783,C_784,C_785,C_786,C_787,C_788,C_789,...,C_5605,C_5606,C_5607,C_5608,C_5609,C_5610,C_5611,C_5612,C_5613,C_5614
SYMBOL,T,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,T,T,T,T,T


In [21]:
Affymetrix_U133 = new_pd.append(Affymetrix_U133)

In [22]:
Affymetrix_U133.head()

CN,C_780,C_781,C_782,C_783,C_784,C_785,C_786,C_787,C_788,C_789,...,C_5605,C_5606,C_5607,C_5608,C_5609,C_5610,C_5611,C_5612,C_5613,C_5614
SYMBOL,T,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,T,T,T,T,T
A2M,15244,21656.5,21376.4,27586.7,11280.8,15715.3,6170.72,28239.3,9387.33,21515.4,...,170.32,488.005,281.183,406.116,89.4406,268.346,517.441,508.63,80.2869,388.483
AACS,3312.27,1020.93,687.675,858.281,1736.04,1318.67,1355.93,638.935,2172.84,1553.79,...,1042.85,672.9,923.001,924.501,1135.67,880.556,758.423,788.322,792.258,600.135
AAK1,666.384,261.522,429.121,261.704,187.638,294.833,429.514,341.132,930.045,528.939,...,748.078,276.208,299.855,276.199,620.601,562.019,377.929,490.828,2076.46,355.813
AAMP,2134.55,2351.49,1849.31,2603.45,2685.11,1907.24,3650.81,2530.55,1799.86,2050.18,...,1738.71,2212.36,1760.63,1712.42,2376.79,2120.28,1668.2,1141.02,3546.35,2025.65


### Объединение платформ

In [23]:
df = pd.merge(Agilent, Affymetrix_U133, left_index=True, right_index=True)

Как и ожидалось, в файле получилось 3091 колонок

In [24]:
df.head()

CN,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_5605,C_5606,C_5607,C_5608,C_5609,C_5610,C_5611,C_5612,C_5613,C_5614
SYMBOL,N,N,N,N,N,N,N,N,N,N,...,T,T,T,T,T,T,T,T,T,T
A2M,7780.57,5649.05,5521.78,11224.4,22027.3,10980.4,6970.97,7455.67,7438.38,3852.8,...,170.32,488.005,281.183,406.116,89.4406,268.346,517.441,508.63,80.2869,388.483
AACS,2077.27,1813.22,1942.76,2063.8,1804.61,1590.92,1642.35,1823.03,2104.44,2105.91,...,1042.85,672.9,923.001,924.501,1135.67,880.556,758.423,788.322,792.258,600.135
AAK1,219.199,223.767,126.759,121.974,162.827,116.405,109.747,139.34,117.042,266.32,...,748.078,276.208,299.855,276.199,620.601,562.019,377.929,490.828,2076.46,355.813
AAMP,3714.61,3209.83,3324.89,3746.21,4941.36,3170.3,3276.38,3154.22,4166.65,5900.79,...,1738.71,2212.36,1760.63,1712.42,2376.79,2120.28,1668.2,1141.02,3546.35,2025.65


#### Переиндекация

In [25]:
df['i'] = [('"' + str(i) + '"')  for i in range(1, df.shape[0] + 1)]
df = df.set_index('i')

In [26]:
df.head()

CN,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_5605,C_5606,C_5607,C_5608,C_5609,C_5610,C_5611,C_5612,C_5613,C_5614
i,,,,,,,,,,,,,,,,,,,,,
"""1""",N,N,N,N,N,N,N,N,N,N,...,T,T,T,T,T,T,T,T,T,T
"""2""",7780.57,5649.05,5521.78,11224.4,22027.3,10980.4,6970.97,7455.67,7438.38,3852.8,...,170.32,488.005,281.183,406.116,89.4406,268.346,517.441,508.63,80.2869,388.483
"""3""",2077.27,1813.22,1942.76,2063.8,1804.61,1590.92,1642.35,1823.03,2104.44,2105.91,...,1042.85,672.9,923.001,924.501,1135.67,880.556,758.423,788.322,792.258,600.135
"""4""",219.199,223.767,126.759,121.974,162.827,116.405,109.747,139.34,117.042,266.32,...,748.078,276.208,299.855,276.199,620.601,562.019,377.929,490.828,2076.46,355.813
"""5""",3714.61,3209.83,3324.89,3746.21,4941.36,3170.3,3276.38,3154.22,4166.65,5900.79,...,1738.71,2212.36,1760.63,1712.42,2376.79,2120.28,1668.2,1141.02,3546.35,2025.65


In [27]:
print(df.index.names)
print(df.columns.names) 

['i']
['CN']


In [28]:
## Убираем название индекса

df = df.rename_axis(None, axis = 1)
df.index.names = [None]

In [29]:
df.head()

,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_5605,C_5606,C_5607,C_5608,C_5609,C_5610,C_5611,C_5612,C_5613,C_5614
"""1""",N,N,N,N,N,N,N,N,N,N,...,T,T,T,T,T,T,T,T,T,T
"""2""",7780.57,5649.05,5521.78,11224.4,22027.3,10980.4,6970.97,7455.67,7438.38,3852.8,...,170.32,488.005,281.183,406.116,89.4406,268.346,517.441,508.63,80.2869,388.483
"""3""",2077.27,1813.22,1942.76,2063.8,1804.61,1590.92,1642.35,1823.03,2104.44,2105.91,...,1042.85,672.9,923.001,924.501,1135.67,880.556,758.423,788.322,792.258,600.135
"""4""",219.199,223.767,126.759,121.974,162.827,116.405,109.747,139.34,117.042,266.32,...,748.078,276.208,299.855,276.199,620.601,562.019,377.929,490.828,2076.46,355.813
"""5""",3714.61,3209.83,3324.89,3746.21,4941.36,3170.3,3276.38,3154.22,4166.65,5900.79,...,1738.71,2212.36,1760.63,1712.42,2376.79,2120.28,1668.2,1141.02,3546.35,2025.65


#### Добавление колонки с нумерацией

In [30]:
df = pd.merge(pd.DataFrame([i for i in range(0, df.shape[0])], columns=['C_0'], 
                           index=[('"' + str(i) + '"')  for i in range(1, df.shape[0] + 1)]), 
              df, left_index=True, right_index=True)

In [31]:
df.head()

,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,...,C_5605,C_5606,C_5607,C_5608,C_5609,C_5610,C_5611,C_5612,C_5613,C_5614
"""1""",0,N,N,N,N,N,N,N,N,N,...,T,T,T,T,T,T,T,T,T,T
"""2""",1,7780.57,5649.05,5521.78,11224.4,22027.3,10980.4,6970.97,7455.67,7438.38,...,170.32,488.005,281.183,406.116,89.4406,268.346,517.441,508.63,80.2869,388.483
"""3""",2,2077.27,1813.22,1942.76,2063.8,1804.61,1590.92,1642.35,1823.03,2104.44,...,1042.85,672.9,923.001,924.501,1135.67,880.556,758.423,788.322,792.258,600.135
"""4""",3,219.199,223.767,126.759,121.974,162.827,116.405,109.747,139.34,117.042,...,748.078,276.208,299.855,276.199,620.601,562.019,377.929,490.828,2076.46,355.813
"""5""",4,3714.61,3209.83,3324.89,3746.21,4941.36,3170.3,3276.38,3154.22,4166.65,...,1738.71,2212.36,1760.63,1712.42,2376.79,2120.28,1668.2,1141.02,3546.35,2025.65


In [32]:
df.columns = [('"V' + str(int(x[2:]) + 1) + '"') for x in df.columns]

In [33]:
df.head()

,"""V1""","""V2""","""V3""","""V4""","""V5""","""V6""","""V7""","""V8""","""V9""","""V10""",...,"""V5606""","""V5607""","""V5608""","""V5609""","""V5610""","""V5611""","""V5612""","""V5613""","""V5614""","""V5615"""
"""1""",0,N,N,N,N,N,N,N,N,N,...,T,T,T,T,T,T,T,T,T,T
"""2""",1,7780.57,5649.05,5521.78,11224.4,22027.3,10980.4,6970.97,7455.67,7438.38,...,170.32,488.005,281.183,406.116,89.4406,268.346,517.441,508.63,80.2869,388.483
"""3""",2,2077.27,1813.22,1942.76,2063.8,1804.61,1590.92,1642.35,1823.03,2104.44,...,1042.85,672.9,923.001,924.501,1135.67,880.556,758.423,788.322,792.258,600.135
"""4""",3,219.199,223.767,126.759,121.974,162.827,116.405,109.747,139.34,117.042,...,748.078,276.208,299.855,276.199,620.601,562.019,377.929,490.828,2076.46,355.813
"""5""",4,3714.61,3209.83,3324.89,3746.21,4941.36,3170.3,3276.38,3154.22,4166.65,...,1738.71,2212.36,1760.63,1712.42,2376.79,2120.28,1668.2,1141.02,3546.35,2025.65


In [34]:
df.loc['"1"', '"V1"'] = "SYMBOL"

In [35]:
df.head()

,"""V1""","""V2""","""V3""","""V4""","""V5""","""V6""","""V7""","""V8""","""V9""","""V10""",...,"""V5606""","""V5607""","""V5608""","""V5609""","""V5610""","""V5611""","""V5612""","""V5613""","""V5614""","""V5615"""
"""1""",SYMBOL,N,N,N,N,N,N,N,N,N,...,T,T,T,T,T,T,T,T,T,T
"""2""",1,7780.57,5649.05,5521.78,11224.4,22027.3,10980.4,6970.97,7455.67,7438.38,...,170.32,488.005,281.183,406.116,89.4406,268.346,517.441,508.63,80.2869,388.483
"""3""",2,2077.27,1813.22,1942.76,2063.8,1804.61,1590.92,1642.35,1823.03,2104.44,...,1042.85,672.9,923.001,924.501,1135.67,880.556,758.423,788.322,792.258,600.135
"""4""",3,219.199,223.767,126.759,121.974,162.827,116.405,109.747,139.34,117.042,...,748.078,276.208,299.855,276.199,620.601,562.019,377.929,490.828,2076.46,355.813
"""5""",4,3714.61,3209.83,3324.89,3746.21,4941.36,3170.3,3276.38,3154.22,4166.65,...,1738.71,2212.36,1760.63,1712.42,2376.79,2120.28,1668.2,1141.02,3546.35,2025.65


### Запись в файл

In [36]:
str_cols = df.columns[df.dtypes.eq('object')]

In [37]:
for index, row in df.iterrows():
    for col in df.columns:
        row[col] = '"' + str(row[col]) + '"'

In [38]:
df.head()

,"""V1""","""V2""","""V3""","""V4""","""V5""","""V6""","""V7""","""V8""","""V9""","""V10""",...,"""V5606""","""V5607""","""V5608""","""V5609""","""V5610""","""V5611""","""V5612""","""V5613""","""V5614""","""V5615"""
"""1""","""SYMBOL""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""",...,"""T""","""T""","""T""","""T""","""T""","""T""","""T""","""T""","""T""","""T"""
"""2""","""1""","""7780.567009364109""","""5649.049966098411""","""5521.78363046514""","""11224.365024506002""","""22027.2538935425""","""10980.4339666004""","""6970.97437690701""","""7455.6722522786295""","""7438.37720975418""",...,"""170.319532493565""","""488.004578450045""","""281.18260236582404""","""406.11620071082297""","""89.440637472567""","""268.346495979736""","""517.440816947885""","""508.63039169064905""","""80.2869210601573""","""388.482510747958"""
"""3""","""2""","""2077.2678844557""","""1813.2230098394""","""1942.7580751891""","""2063.80472591412""","""1804.6127234029602""","""1590.9175645993803""","""1642.35447387543""","""1823.0250004477803""","""2104.4438842997197""",...,"""1042.84968672749""","""672.899860788046""","""923.0008933678281""","""924.501364089821""","""1135.67402000521""","""880.5560498274541""","""758.422583134949""","""788.322474283661""","""792.258138452522""","""600.134995040699"""
"""4""","""3""","""219.198827716941""","""223.767088796953""","""126.758978930383""","""121.974332565274""","""162.827446713364""","""116.404527338655""","""109.74688181825""","""139.339870682355""","""117.041910940336""",...,"""748.0781446486079""","""276.207751143599""","""299.855225240103""","""276.199409222242""","""620.600524212345""","""562.01882187285""","""377.92904477064906""","""490.82798287762904""","""2076.45767978887""","""355.812547802643"""
"""5""","""4""","""3714.6129894804""","""3209.82623040731""","""3324.89047260802""","""3746.21365290628""","""4941.36455775631""","""3170.2993716790397""","""3276.3750935617104""","""3154.22056478817""","""4166.65244651529""",...,"""1738.7115518049802""","""2212.3598584239103""","""1760.62600953742""","""1712.42124014625""","""2376.7942024282""","""2120.27815779072""","""1668.2014134069""","""1141.02062963145""","""3546.3471293082102""","""2025.6453510950698"""


In [39]:
df.tail()

,"""V1""","""V2""","""V3""","""V4""","""V5""","""V6""","""V7""","""V8""","""V9""","""V10""",...,"""V5606""","""V5607""","""V5608""","""V5609""","""V5610""","""V5611""","""V5612""","""V5613""","""V5614""","""V5615"""
"""4587""","""4586""","""77.7591530534735""","""99.8969212391094""","""60.658294767816""","""75.2781289645424""","""85.8361372869267""","""54.7655434289893""","""57.582987193730204""","""110.707648630859""","""74.68149235511359""",...,"""52.25987930039061""","""55.1504067206748""","""45.9789610114389""","""36.2828503835861""","""32.0892958594654""","""126.431985331393""","""45.372118680392""","""36.7506648933132""","""49.585927369142""","""144.555015088519"""
"""4588""","""4587""","""800.39124665035""","""893.767240884558""","""575.5641453500799""","""723.263625309091""","""927.75227570071""","""755.294704896225""","""647.119313392501""","""792.421654544984""","""853.1081592629849""",...,"""3878.4279524953004""","""926.470198032946""","""2916.85292790025""","""2863.8808507383196""","""4103.75863262484""","""5091.90002940056""","""3204.21181206605""","""4384.8665102956""","""4309.08578386352""","""1542.5383924217801"""
"""4589""","""4588""","""2716.96822637138""","""1491.16202475034""","""2825.84311795226""","""2955.38559064118""","""7073.764349097441""","""2110.8750873584""","""2627.62337329276""","""1544.89749576133""","""2358.96224363859""",...,"""5357.509550402539""","""26610.330611829""","""12822.050534261001""","""14728.671700363999""","""8441.67168755831""","""10106.721436525""","""25985.2569953846""","""18019.8698151617""","""41836.840803154904""","""9835.688339175891"""
"""4590""","""4589""","""2387.33014194705""","""2294.32834922892""","""1641.23313186272""","""1784.91336717837""","""2040.3303963953301""","""1718.94048684103""","""1566.45713814708""","""2060.03177657151""","""1688.62756275972""",...,"""1831.3601646494199""","""1318.64289613602""","""1450.73459327516""","""1983.9447247077899""","""1795.81060714226""","""1804.76185816728""","""1190.77528866879""","""2254.4197802742497""","""1411.95530643948""","""1483.4988751492401"""
"""4591""","""4590""","""860.5489178281799""","""1257.86304997661""","""1939.69543015852""","""870.6637960954539""","""1269.10230144281""","""1408.7783018474402""","""1601.4974193754401""","""1424.20453504142""","""1140.02152223719""",...,"""2954.3702225091197""","""3295.75462925818""","""2908.0513376595""","""3696.0943713829497""","""3494.5895548789003""","""4325.682937990179""","""3015.7246535350396""","""5619.815369720351""","""4498.64908143525""","""3119.5930750771804"""


In [40]:
import csv

df.to_csv('50.txt', header=True, index=True, sep=' ', quoting=csv.QUOTE_NONE)

In [41]:
check_data = pd.read_table('50.txt',sep='\s+', engine='python')

In [42]:
check_data.head()

,"""V1""","""V2""","""V3""","""V4""","""V5""","""V6""","""V7""","""V8""","""V9""","""V10""",...,"""V5606""","""V5607""","""V5608""","""V5609""","""V5610""","""V5611""","""V5612""","""V5613""","""V5614""","""V5615"""
"""1""","""SYMBOL""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""",...,"""T""","""T""","""T""","""T""","""T""","""T""","""T""","""T""","""T""","""T"""
"""2""","""1""","""7780.567009364109""","""5649.049966098411""","""5521.78363046514""","""11224.365024506002""","""22027.2538935425""","""10980.4339666004""","""6970.97437690701""","""7455.6722522786295""","""7438.37720975418""",...,"""170.319532493565""","""488.004578450045""","""281.18260236582404""","""406.11620071082297""","""89.440637472567""","""268.346495979736""","""517.440816947885""","""508.63039169064905""","""80.2869210601573""","""388.482510747958"""
"""3""","""2""","""2077.2678844557""","""1813.2230098394""","""1942.7580751891""","""2063.80472591412""","""1804.6127234029602""","""1590.9175645993803""","""1642.35447387543""","""1823.0250004477803""","""2104.4438842997197""",...,"""1042.84968672749""","""672.899860788046""","""923.0008933678281""","""924.501364089821""","""1135.67402000521""","""880.5560498274541""","""758.422583134949""","""788.322474283661""","""792.258138452522""","""600.134995040699"""
"""4""","""3""","""219.198827716941""","""223.767088796953""","""126.758978930383""","""121.974332565274""","""162.827446713364""","""116.404527338655""","""109.74688181825""","""139.339870682355""","""117.041910940336""",...,"""748.0781446486079""","""276.207751143599""","""299.855225240103""","""276.199409222242""","""620.600524212345""","""562.01882187285""","""377.92904477064906""","""490.82798287762904""","""2076.45767978887""","""355.812547802643"""
"""5""","""4""","""3714.6129894804""","""3209.82623040731""","""3324.89047260802""","""3746.21365290628""","""4941.36455775631""","""3170.2993716790397""","""3276.3750935617104""","""3154.22056478817""","""4166.65244651529""",...,"""1738.7115518049802""","""2212.3598584239103""","""1760.62600953742""","""1712.42124014625""","""2376.7942024282""","""2120.27815779072""","""1668.2014134069""","""1141.02062963145""","""3546.3471293082102""","""2025.6453510950698"""
